In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
tf.__version__

'2.15.0'

In [3]:
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [6]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) #convert all the text to indexes

def split_input_target(chunk): #split a sequence of text to go 1 character shift. eg: First => input: Fir, output: Irs
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [9]:
seq_length = 50 #the ngram length of input and output features, for eg. First -> Fir,Irs
len(text_as_int)

1115394

In [10]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
char_dataset.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=1115394>

In [11]:
for i in char_dataset.take(15):
    print(idx2char[i.numpy()],end=" ")

F i r s t   C i t i z e n : 
 

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

for inputer,targeter in dataset.take(3):
    print("[inputer]",''.join(idx2char[inputer.numpy()]),"\[ntargeter]",''.join(idx2char[targeter.numpy()]))

[inputer] First Citizen:
Before we proceed any further, hear \[ntargeter] irst Citizen:
Before we proceed any further, hear 
[inputer] me speak.

All:
Speak, speak.

First Citizen:
You  \[ntargeter] e speak.

All:
Speak, speak.

First Citizen:
You a
[inputer] re all resolved rather to die than to famish?

All \[ntargeter] e all resolved rather to die than to famish?

All:


In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 50), dtype=tf.int64, name=None), TensorSpec(shape=(64, 50), dtype=tf.int64, name=None))>

In [14]:
''.join(char2idx.keys())

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [15]:
vocab_size = len(vocab)
embedding_dim = 128 #GRU embed
rnn_units = 384 #RNN units
batch_size = 64

In [16]:
checkpoint_dir = './training_checkpoints'

In [40]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            reset_after=False,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])

In [42]:
ckpt_number = 9
model.load_weights(f"{checkpoint_dir}/ckpt_{ckpt_number}.weights.h5")

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape,
          "# (batch_size, sequence_length, vocab_size)")

(64, 50, 65) # (batch_size, sequence_length, vocab_size)


In [35]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (64, None, 128)           8320      
                                                                 
 gru_6 (GRU)                 (64, None, 384)           590976    
                                                                 
 dense_6 (Dense)             (64, None, 65)            25025     
                                                                 
Total params: 624321 (2.38 MB)
Trainable params: 624321 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
def criterion(labels,logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=criterion) 

In [ ]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

In [38]:
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
EPOCHS=10

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10


341/341 [==============================] - 430s 1s/step - loss: 2.5457
Epoch 2/10
341/341 [==============================] - 426s 1s/step - loss: 1.9314
Epoch 3/10
341/341 [==============================] - 431s 1s/step - loss: 1.7235
Epoch 4/10
341/341 [==============================] - 433s 1s/step - loss: 1.6120
Epoch 5/10
341/341 [==============================] - 435s 1s/step - loss: 1.5462
Epoch 6/10
341/341 [==============================] - 436s 1s/step - loss: 1.5001
Epoch 7/10
341/341 [==============================] - 437s 1s/step - loss: 1.4678
Epoch 8/10
341/341 [==============================] - 434s 1s/step - loss: 1.4405
Epoch 9/10
341/341 [==============================] - 444s 1s/step - loss: 1.4206
Epoch 10/10
341/341 [==============================] - 428s 1s/step - loss: 1.4013


In [17]:
ckpt_number=10
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[1, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            reset_after=False,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
model.build(tf.TensorShape([1, None]))
model.load_weights(f"{checkpoint_dir}/ckpt_{ckpt_number}.weights.h5")

In [18]:
start_string=u"An"
input_eval = [char2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

predictions = model(input_eval)
predictions = tf.squeeze(predictions, 0)
idx2char[tf.random.categorical(predictions, 1)[-1, 0].numpy()]

'd'

In [19]:
def generate_text(model, start_string):
    num_generate = 300
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 0.25

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(
            predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [20]:
print(generate_text(model, start_string=u"ROMEO:"))

ROMEO:
I have not to be sent to hear thee of the state of the charge of the heart is not to the good father with him.

PETRUCHIO:
What says the lady since with the manner of the common profane to the prince,
That the first of the prince of the traitor to the prince,
That with him to him to the prince seem


In [49]:
model.save("model.h5") #NOTE: .keras files DONT work,  --input_format keras for converter <=4.15 and monkeypatch site-packages/tensorflowjs/converters/jax_conversion.py 

/Users/bitan/Desktop/poppedcorn/dist/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
model.save("saved_model",save_format="tf",save_traces=True)

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [25]:
import tensorflowjs as tfjs

In [27]:
tfjs.converters.save_keras_model(model,"makachu")

/Users/bitan/Desktop/poppedcorn/dist/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
